## Load video and gloss data, and prepare for classification

1 load the segmented gloss videos

2 extract labels for data; get data statistics

3 split videos into gloss chunks

3.1 turn those gloss chunks into frames and save frames as array

4 create sentences dataframe and dataframe of frame arrays

In [1]:
#1
#!pip install ffmpeg
#!pip install Pillow

from PIL import Image
import os
import shutil
import pandas as pd
import ffmpeg
import numpy as np
from numpy import asarray
import cv2 as cv
import re
import math
from subprocess import check_call, PIPE, Popen
import shlex
from tqdm import tqdm



---



## Get statistics for ECHO and CorpusNGT:

First we have to split the CorpusNGT videos by speaker

In [11]:
def cleanItemStrict(item):
    if "(p-)" in item:  item = item.replace("(p-)", "")
    if "(g-)" in item:  item = item.replace("(g-)", "")
    if "/" in item:  item = item.split("/")[0] # make sure the gloss name is regularized:
    if " (" in item:  item = item.split(" (")[0]
    if ") " in item:  item = item.split(") ")[1]
    if ")-" in item:  item = item.split(")-")[1]
    if " " in item:  item = item.replace(" ", "")
    if '"' in item:  item = item.replace('"', "")
    if "'" in item:  item = item.replace("'", "")
    if "~" in item:  item = item.replace("~", "")
    if "." in item:  item = item.replace(".", "-")
    if "_" in item:  item = item.replace("_", "-")
    if "?" in item:  item = item.replace("?", "")
    if ">" in item:  item = item.replace(">", "")
    if "<" in item:  item = item.replace("<", "")
    if "%" in item:  item = item.replace("%", "")
    if "#" in item:  item = item.replace("#", "")
    if "!" in item:  item = item.replace("!", "")
    if "$" in item:  item = item.replace("$", "")
    if "*" in item:  item = item.replace("*", "")
    if "&" in item:  item = item.replace("&", "")
    if "+" in item:  item = item.replace("+", "")
    if "`" in item:  item = item.replace("`", "")
    if "-1" in item:  item = item.replace("-1", "")
    if "-2" in item:  item = item.replace("-2", "")
    if "-3" in item:  item = item.replace("-3", "")
    if "-4" in item:  item = item.replace("-4", "")
    if "-5" in item:  item = item.replace("-5", "")
    if "-6" in item:  item = item.replace("-6", "")
    if "-7" in item:  item = item.replace("-7", "")
    if "-8" in item:  item = item.replace("-8", "")
    if "-9" in item:  item = item.replace("-9", "")
    if "-0" in item:  item = item.replace("-0", "")
    if "-EC" in item:  item = item.split("-EC")[0]
    if "-HC" in item:  item = item.split("-HC")[0]
    if "+PT" in item:  item = item.replace("+PT", "")
    if "+PO" in item:  item = item.replace("+PO", "")
    if "-RC" in item:  item = item.replace("-RC", "")
    if "-A" in item:  item = item.replace("-A", "")
    if "-B" in item:  item = item.replace("-B", "")
    if "-C" in item:  item = item.replace("-C", "")
    if "-D" in item:  item = item.replace("-D", "")
    if "-E" in item:  item = item.replace("-E", "")
    if "-F" in item:  item = item.replace("-F", "")
    if "-G" in item:  item = item.replace("-G", "")
    if "-H" in item:  item = item.replace("-H", "")
    if "(H)" in item:  item = item.replace("(H)", "")
    if "(-H)" in item:  item = item.replace("(-H)", "")
    if "(2H)" in item:  item = item.replace("(2H)", "")
    item = item.upper()
    return(item)

def cleanItem(series):
    output = []
    for item in series:
        item = cleanItemPunct(item)
        output.append(item)
    return output

def cleanItemPunct(item):
    if "/" in item:  item = item.split("/")[0] # make sure the gloss name is regularized:
    if " (" in item:  item = item.split(" (")[0]
    if ") " in item:  item = item.split(") ")[1]
    if ")-" in item:  item = item.split(")-")[1]
    if " " in item:  item = item.replace(" ", "")
    if '"' in item:  item = item.replace('"', "")
    if "'" in item:  item = item.replace("'", "")
    if "~" in item:  item = item.replace("~", "")
    if "." in item:  item = item.replace(".", "-")
    if "_" in item:  item = item.replace("_", "-")
    if "?" in item:  item = item.replace("?", "")
    if ">" in item:  item = item.replace(">", "")
    if "<" in item:  item = item.replace("<", "")
    if "%" in item:  item = item.replace("%", "")
    if "#" in item:  item = item.replace("#", "")
    if "!" in item:  item = item.replace("!", "")
    if "$" in item:  item = item.replace("$", "")
    if "*" in item:  item = item.replace("*", "")
    if "&" in item:  item = item.replace("&", "")
    if "+" in item:  item = item.replace("+", "")
    if "`" in item:  item = item.replace("`", "")
    if ":" in item:  item = item.replace(":", "-")
    if ";" in item:  item = item.replace(";", "-")
    if "," in item:  item = item.replace(",", "-")
    if "[" in item:  item = item.replace("[", "")
    if "]" in item:  item = item.replace("]", "")
    item = item.upper()
    return(item)

In [12]:
ngtglossdir = "Data/CorpusNGT/gloss/"

for filename in os.listdir(ngtglossdir):
    f = os.path.join(ngtglossdir, filename)
    file, extension = filename.split('.')
    # For the csv file, we get all the statistics like gloss count etc
    if extension == 'mpg':
        os.rename('{}{}.mpg'.format(ngtglossdir, file), '{}{}.mpg'.format(ngtglossdir, file.replace('_b', "")))

keepvideofiles = []

for filename in os.listdir(ngtglossdir):
    f = os.path.join(ngtglossdir, filename)
    file, extension = filename.split('.')
    # For the csv file, we get all the statistics like gloss count etc

    if extension == 'csv':
        #if "_S" not in file:
        gloss = pd.read_csv(f) #, header=None, names = ['signer', 'start time', 'start seconds', 'stop time', 'stop seconds', 'duration', 'value', 'gloss']
        gloss = gloss.dropna(axis=0)
        #for item in gloss['gloss']:
        #    item = cleanItemPunct(item) #cleanItemStrict(item)

        gloss['gloss'] = cleanItem(gloss['gloss'])
        
        gloss = gloss.dropna(axis=0)
        signers = gloss['signer'].unique()
        #gloss = gloss.set_index('signer', drop=True, append=False)

        if len(signers) == 0:
            print("No speakers in file, check absence of speakers for file " + file)
        elif len(signers) > 0:
            #S1_number = signers[0]
            try:
                #gloss_S1 = gloss.loc[[S1_number]]
                gloss.to_csv('{}{}.csv'.format(ngtglossdir, file))
                keepvideofiles.append('{}{}.mpg'.format(ngtglossdir, file))
            except:
                print("Error with file: " + file + ", Check manually")
                #try:  os.remove('{}{}_{}.mpg'.format(ngtglossdir, file, S1_number))
                #except: pass
        
        #os.remove('{}{}.csv'.format(ngtglossdir, file))

In [13]:
print(len(keepvideofiles))
print(keepvideofiles)

604
['Data/CorpusNGT/gloss/CNGT0004_S003.mpg', 'Data/CorpusNGT/gloss/CNGT0004_S004.mpg', 'Data/CorpusNGT/gloss/CNGT0005_S003.mpg', 'Data/CorpusNGT/gloss/CNGT0005_S004.mpg', 'Data/CorpusNGT/gloss/CNGT0006_S003.mpg', 'Data/CorpusNGT/gloss/CNGT0006_S004.mpg', 'Data/CorpusNGT/gloss/CNGT0007_S003.mpg', 'Data/CorpusNGT/gloss/CNGT0007_S004.mpg', 'Data/CorpusNGT/gloss/CNGT0008_S003.mpg', 'Data/CorpusNGT/gloss/CNGT0008_S004.mpg', 'Data/CorpusNGT/gloss/CNGT0009_S003.mpg', 'Data/CorpusNGT/gloss/CNGT0009_S004.mpg', 'Data/CorpusNGT/gloss/CNGT0010_S003.mpg', 'Data/CorpusNGT/gloss/CNGT0010_S004.mpg', 'Data/CorpusNGT/gloss/CNGT0011_S003.mpg', 'Data/CorpusNGT/gloss/CNGT0011_S004.mpg', 'Data/CorpusNGT/gloss/CNGT0012_S003.mpg', 'Data/CorpusNGT/gloss/CNGT0012_S004.mpg', 'Data/CorpusNGT/gloss/CNGT0013_S003.mpg', 'Data/CorpusNGT/gloss/CNGT0013_S004.mpg', 'Data/CorpusNGT/gloss/CNGT0014_S003.mpg', 'Data/CorpusNGT/gloss/CNGT0014_S004.mpg', 'Data/CorpusNGT/gloss/CNGT0015_S003.mpg', 'Data/CorpusNGT/gloss/CNGT001

In [14]:
VideosRemovedCounter = 0
for filename in os.listdir(ngtglossdir):
    f = os.path.join(ngtglossdir, filename)
    if f.split(".")[1] == "mpg":
        if f not in keepvideofiles:
            os.remove(f)
            VideosRemovedCounter += 1

print(VideosRemovedCounter)

0


In [15]:
!pip install moviepy

from moviepy.video.io.VideoFileClip import VideoFileClip

In [16]:
total_unique_glosses = dict()

def getStats(directorygloss, directorytrans, trans=True, CorpusNGT=False):
    # Gets statistics like average video duration, number of translated videos, number of unique glosses, number of sentences, avg number of sentences etc
    # for every video in the ECHO and CorpusNGT corpus.

    unique_glosses = dict()
    gloss_count = dict()
    translated_video_count = 0
    glossed_video_count = 0
    sentence_count = 0
    video_lengths = []
    video_gloss_counts = []

    if trans:
        for filename in os.listdir(directorytrans):
            f = os.path.join(directorytrans, filename)
            file, extension = filename.split('.')

            # For the csv file, we get all the statistics like gloss count etc
            if extension == 'csv':
                translated_video_count += 1
                glossed_video_count += 1
                gloss = pd.read_csv(f, header=None, names = ['signer', 'start time', 'start seconds', 'stop time', 'stop seconds', 'value', 'gloss'])
                gloss = gloss.dropna(axis=0)

                try:
                    sentences = gloss.loc[["Translation Dutch"]]
                    sentence_count += len(sentences)
                except:
                    pass
                try:
                    sentences = gloss.loc[["Dutch Translation"]]
                    sentence_count += len(sentences)
                except:
                    pass

    for filename in os.listdir(directorygloss):
        f = os.path.join(directorygloss, filename)
        file, extension = filename.split('.')
        # For the csv file, we get all the statistics like gloss count etc

        if extension == 'csv':
            glosses_in_video = 0
            glossed_video_count += 1
            gloss = pd.read_csv(f, header=None, names = ['signer', 'start time', 'start seconds', 'stop time', 'stop seconds', 'value', 'gloss'])
            gloss = gloss.dropna(axis=0)

            if not CorpusNGT:
                glossRH = gloss.loc["Gloss RH Dutch"]
                glossLH = gloss.loc["Gloss LH Dutch"]
                gloss = pd.concat([glossRH, glossLH])

            for item in gloss['gloss']:
                
                item = cleanItem(item)
                item = ''.join(item)
                
                if item == '':
                    continue
                glosses_in_video += 1
                if item not in unique_glosses:
                    n = len(unique_glosses)
                    unique_glosses[item] = n
                    if item not in total_unique_glosses:
                        n = len(total_unique_glosses)
                        total_unique_glosses[item] = n
                    gloss_count[item] = 0
                gloss_count[item] = gloss_count[item]+1
            video_gloss_counts.append(glosses_in_video)

        if extension == 'mpg':
            video = VideoFileClip(f)
            duration = video.duration
            video_lengths.append(duration)
            video.reader.close()

    average_video_gloss_count = sum(video_gloss_counts)/len(video_gloss_counts)
    average_video_length = sum(video_lengths)/len(video_lengths)
    return(translated_video_count, glossed_video_count, unique_glosses, gloss_count, average_video_length, sentence_count, average_video_gloss_count)

def writeStats(translated_video_count, glossed_video_count, unique_glosses, gloss_count, average_video_length, sentence_count, average_video_gloss_count, name='data_statistics.txt', CorpusNGT=False):
    with open('Data/' + name, 'w') as f:
        f.write("Nr of glossed videos: \t\t\t" + str(glossed_video_count) + "\n")
        f.write("Nr of unique glosses: \t\t\t" + str(len(unique_glosses)) + "\n")
        f.write("Nr of total glosses: \t\t\t" + str(sum(gloss_count.values())) + "\n")
        f.write("Average length of videos: \t\t" + str(average_video_length) + " sec or " + str(average_video_length/60) + " min" + "\n")
        f.write("Avg number of glosses per video: \t" + str(average_video_gloss_count) + "\n")
        if not CorpusNGT:
            f.write("Avg number of glosses per sentence: \t" + str(average_video_gloss_count/(sentence_count/translated_video_count)) + "\n")
            f.write("Avg number of sentences per video: \t" + str(sentence_count/translated_video_count) + "\n")
            f.write("Nr of sentences: \t\t\t" + str(sentence_count) + "\n")
            f.write("Nr of translated videos: \t\t" + str(translated_video_count) + "\n")

glossdir = "Data/ECHO/gloss/"
glosstransdir = "Data/ECHO/gloss_trans/"

translated_video_count, glossed_video_count, e_unique_glosses, e_gloss_count, average_video_length, sentence_count, average_video_gloss_count = getStats(glossdir, glosstransdir)
writeStats(translated_video_count, glossed_video_count, e_unique_glosses, e_gloss_count, average_video_length, sentence_count, average_video_gloss_count, name='data_statistics_ECHO.txt')

print("Nr of translated videos in ECHO: \t\t" + str(translated_video_count))
print(("Nr of glossed videos in ECHO: \t\t\t" + str(glossed_video_count)))
print("Nr of unique glosses in ECHO: \t\t\t" + str(len(e_unique_glosses)))
print("Nr of total glosses in ECHO: \t\t\t" + str(sum(e_gloss_count.values())))
print("Nr of sentences in ECHO: \t\t\t" + str(sentence_count))
print("Average length of videos in ECHO: \t\t" + str(average_video_length) + " sec or " + str(average_video_length/60) + " min")
print("Avg number of sentences per video in ECHO: \t" + str(sentence_count/translated_video_count))
print("Avg number of glosses per video in ECHO: \t" + str(average_video_gloss_count))
print("Avg number of glosses per sentence in ECHO: \t" + str(average_video_gloss_count/(sentence_count/translated_video_count)))
print("Count for each gloss in ECHO: \t\t\t" + str(e_gloss_count))
print()

glossdir = "Data/CorpusNGT/gloss/"

translated_video_count, glossed_video_count, c_unique_glosses, c_gloss_count, average_video_length, sentence_count, average_video_gloss_count = getStats(glossdir, glosstransdir, trans=False, CorpusNGT=True)
writeStats(translated_video_count, glossed_video_count, c_unique_glosses, c_gloss_count, average_video_length, sentence_count, average_video_gloss_count, name='data_statistics_CorpusNGT.txt', CorpusNGT=True)

print(("Nr of glossed videos in CNGT: \t\t\t" + str(glossed_video_count)))
print("Nr of unique glosses in CNGT: \t\t\t" + str(len(c_unique_glosses)))
print("Nr of total glosses in CNGT: \t\t\t" + str(sum(c_gloss_count.values())))
print("Average length of videos in CNGT: \t\t" + str(average_video_length) + " sec or " + str(average_video_length/60) + " min")
print("Avg number of glosses per video in CNGT: \t" + str(average_video_gloss_count))
print("Count for each gloss in CNGT: \t\t\t" + str(c_gloss_count))

Nr of translated videos in ECHO: 		16
Nr of glossed videos in ECHO: 			20
Nr of unique glosses in ECHO: 			465
Nr of total glosses in ECHO: 			1141
Nr of sentences in ECHO: 			614
Average length of videos in ECHO: 		619.5733333333334 sec or 10.326222222222222 min
Avg number of sentences per video in ECHO: 	38.375
Avg number of glosses per video in ECHO: 	285.25
Avg number of glosses per sentence in ECHO: 	7.4332247557003255
Count for each gloss in ECHO: 			{'IND': 40, 'DORP': 5, 'LOCATIEINDIEREGIO': 1, 'AANWEZIG': 7, 'ÉÉN': 2, 'JONGEN': 1, 'HERDER': 7, 'ELKE': 2, 'MORGEN': 1, 'SCHAAP': 4, 'VAN': 1, 'MENSEN': 9, 'INDLOCATIE': 1, 'OPHALEN': 1, 'BIJ-ELKAAR-BRENGEN': 2, 'HEUVEL': 2, 'BASISHAND': 2, 'KUDDEDEHEUVELOP': 2, 'PLUS': 3, 'AVOND': 6, 'KUDDENAARBENEDEN': 1, 'TERUGNAARDEMENSENINHETDORP': 1, '(G-)PU': 14, 'BLIJVEN': 1, 'SCHAPEN-DRIJVEN': 1, 'SOMS': 3, 'TIJD': 4, 'ONTSPANN': 1, 'KIJKEN': 8, 'MOOI': 2, 'MAAR': 7, 'ELKE-DAG': 1, 'HELE': 1, 'DAG': 6, 'GRAZEN': 1, 'GENOEG-HEBBEN': 1, 'LAT

In [17]:
# Save dict of unique glosses and labels
np.save('Data/CorpusNGT/gloss_labels.npy', c_unique_glosses)
np.save('Data/ECHO/gloss_labels.npy', e_unique_glosses)

In [18]:
np.save('Data/CorpusNGT/gloss_counts.npy', c_gloss_count)
np.save('Data/ECHO/gloss_counts.npy', e_gloss_count)

## Preprocess all ECHO data and CorpusNGT

Meaning, we split the videos into gloss sized chunks, and we create a csv file with all the glosses matched to sentences

In [19]:
c_unique_glosses = np.load('Data/CorpusNGT/gloss_labels.npy', allow_pickle='TRUE').item()
#print(read_dictionary['WANDELEN']) # displays "world"

In [20]:
print(c_unique_glosses)

{'GLOSS': 0, 'LANG-B': 1, 'PO': 2, 'ZONDAG-A': 3, 'FIETSEN': 4, 'LOPEN-D': 5, 'WATER-A': 6, 'SPRINGEN-B': 7, 'STAAN-D': 8, 'BEEN-B': 9, 'GEK-A': 10, 'HOND-A': 11, 'SPRINGEN-A': 12, 'DUWEN': 13, 'MOVE-EC-5S': 14, 'MEE': 15, 'MOVE-EC-5': 16, 'GOOIEN': 17, 'KLAAR-A': 18, 'KLEREN-B': 19, 'VOCHTIG': 20, 'LACHEN-A': 21, 'UITKLEDEN-B': 22, 'VAN-A': 23, 'BROEK': 24, 'AANKLEDEN-A': 25, 'AL': 26, 'WACHTEN-A': 27, 'DROOG-A': 28, 'AANKLEDEN-B': 29, 'WEG-C': 30, 'MOVE-EC-1': 31, 'BROOD-B': 32, 'JAM-A': 33, 'ONDERWIJS': 34, 'INTERNAAT-C': 35, 'ONGESTELD': 36, 'VERSCHILLENDE-A': 37, 'DUS-A': 38, 'ZENUWACHTIG-D': 39, 'OPGELUCHT-A': 40, 'BETER': 41, 'VERSLAAN': 42, 'GOED-B': 43, 'LOPEN-B': 44, 'PANTOFFELS': 45, 'BE-EC-B': 46, 'UUR-7-B': 47, 'BINNEN': 48, 'MOETEN-A': 49, 'LAATSTE-A': 50, 'PT': 51, 'MOOI-A': 52, 'HANDEN-WRIJVEN-B': 53, 'SCHRIKKEN-B': 54, 'SCHRIKKEN-A': 55, 'SCHREEUWEN-A': 56, 'VLUCHTEN': 57, 'KAPOT': 58, 'BE-EC-4': 59, 'HOUDEN-VAN': 60, 'SCHOOL-C': 61, 'ZATERDAG-B': 62, 'KAART-A': 63, 'B

In [21]:
import collections
od = collections.OrderedDict(sorted(c_unique_glosses.items()))

In [22]:
print(od)

OrderedDict([('0-A', 2042), ('0-B', 1327), ('0-C', 2683), ('1-A', 121), ('10-A', 1289), ('10-B', 1230), ('100', 3746), ('100-A', 442), ('100-B', 3456), ('1000-A', 1694), ('1000-B', 1708), ('11-B', 1176), ('11-C', 3214), ('11-D', 3368), ('12-B', 2305), ('12-C', 3115), ('12-D', 3369), ('13-A', 3218), ('13-B', 1177), ('14-A', 1632), ('14-B', 2348), ('14-C', 3717), ('15-A', 3212), ('15-B', 2475), ('15-C', 1352), ('158', 3629), ('16-A', 3370), ('16-B', 1353), ('17-A', 3213), ('17-B', 2537), ('18-A', 884), ('18-B', 3219), ('18-C', 1501), ('19-A', 3620), ('19-B', 2377), ('19-C', 3451), ('2-A', 319), ('2-B', 1645), ('2-ORD', 461), ('20-A', 520), ('20-B', 166), ('22', 612), ('26', 2378), ('3-A', 1033), ('3-B', 628), ('3-ORD-A', 518), ('3-ORD-B', 3514), ('30-A', 1386), ('30-B', 2796), ('30-C', 1620), ('4-A', 258), ('4-ORD', 2335), ('40-A', 2522), ('40-B', 2797), ('40-C', 2805), ('5', 438), ('50', 3748), ('50-A', 1310), ('50-B', 2798), ('50-C', 1621), ('500', 1864), ('500^', 3140), ('6-A', 424), 

In [22]:
# first we create a folder for the gloss videos
# for each unique gloss we make a folder in our videos directory, and in our images directory
viddir_name = "Data/ECHO/gloss_split"
for i in e_unique_glosses:
    os.mkdir(os.path.join(viddir_name , str(i)))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Data/ECHO/gloss_split\\IND'

In [3]:
# first we create a folder for the gloss videos
# for each unique gloss we make a folder in our videos directory, and in our images directory
viddir_name = "Data/CorpusNGT/gloss_split_8frames"
for i in c_unique_glosses:
    try:
        os.mkdir(os.path.join(viddir_name , str(i)))
    except:
        print("folder already exists")

NameError: name 'c_unique_glosses' is not defined

In [33]:
## Define all functions again with alterations

re_metadata = re.compile('Duration: (\d{2}):(\d{2}):(\d{2})\.\d+,.*\n.* (\d+(\.\d+)?) fps')

def get_metadata(filename):
    '''
    Get video metadata using ffmpeg
    '''
    p1 = Popen(["ffmpeg", "-hide_banner", "-i", filename], stderr=PIPE, universal_newlines=True)
    output = p1.communicate()[1]
    print(output)
    matches = re_metadata.search(output)
    if matches:
        video_length = int(matches.group(1)) * 3600 + int(matches.group(2)) * 60 + int(matches.group(3))
        video_fps = float(matches.group(4))
        # print('video_length = {}\nvideo_fps = {}'.format(video_length, video_fps))
    else:
        raise Exception("Can't parse required metadata")
    return video_length, video_fps

def split_cut(path, outputpath, filename, ext, starts, stops, glosses, gloss_counter):
    '''
    Split video by cutting and re-encoding: accurate but very slow
    Adding "-c copy" speed up the process but causes imprecise chunk durations
    Reference: https://stackoverflow.com/a/28884437/1862500
    '''
    split_count = len(starts)
    input = '{}{}.{}'.format(path, filename, ext)

    # parse meta data
    video_length, video_fps = get_metadata(input)

    # split the video by gloss
    output = []

    for i in range(split_count):
        split_start = float(starts[i]-0.05)
        split_size = float(stops[i]+0.05) - float(starts[i])
        counter = gloss_counter[glosses[i]]
        output_path = '{}{}/{}.{}'.format(outputpath, glosses[i], counter, ext)
        gloss_counter[glosses[i]] = gloss_counter[glosses[i]]+1
        cmd = 'ffmpeg -hide_banner -loglevel panic -ss {} -t {} -i "{}" -y "{}"'.format(
            split_start,
            split_size,
            input,
            output_path
        )
        # print(cmd)
        check_call(shlex.split(cmd), universal_newlines=True)
        output.append(output_path)
    return output

In [34]:
print(c_unique_glosses)

{'GLOSS': 0, 'LANG-B': 1, 'PO': 2, 'ZONDAG-A': 3, 'FIETSEN': 4, 'LOPEN-D': 5, 'WATER-A': 6, 'SPRINGEN-B': 7, 'STAAN-D': 8, 'BEEN-B': 9, 'GEK-A': 10, 'HOND-A': 11, 'SPRINGEN-A': 12, 'DUWEN': 13, 'MOVE-EC-5S': 14, 'MEE': 15, 'MOVE-EC-5': 16, 'GOOIEN': 17, 'KLAAR-A': 18, 'KLEREN-B': 19, 'VOCHTIG': 20, 'LACHEN-A': 21, 'UITKLEDEN-B': 22, 'VAN-A': 23, 'BROEK': 24, 'AANKLEDEN-A': 25, 'AL': 26, 'WACHTEN-A': 27, 'DROOG-A': 28, 'AANKLEDEN-B': 29, 'WEG-C': 30, 'MOVE-EC-1': 31, 'BROOD-B': 32, 'JAM-A': 33, 'ONDERWIJS': 34, 'INTERNAAT-C': 35, 'ONGESTELD': 36, 'VERSCHILLENDE-A': 37, 'DUS-A': 38, 'ZENUWACHTIG-D': 39, 'OPGELUCHT-A': 40, 'BETER': 41, 'VERSLAAN': 42, 'GOED-B': 43, 'LOPEN-B': 44, 'PANTOFFELS': 45, 'BE-EC-B': 46, 'UUR-7-B': 47, 'BINNEN': 48, 'MOETEN-A': 49, 'LAATSTE-A': 50, 'PT': 51, 'MOOI-A': 52, 'HANDEN-WRIJVEN-B': 53, 'SCHRIKKEN-B': 54, 'SCHRIKKEN-A': 55, 'SCHREEUWEN-A': 56, 'VLUCHTEN': 57, 'KAPOT': 58, 'BE-EC-4': 59, 'HOUDEN-VAN': 60, 'SCHOOL-C': 61, 'ZATERDAG-B': 62, 'KAART-A': 63, 'B

In [44]:
## Here I split all the glossed videos by gloss and save as gloss chunks for CorpusNGT

datapath = "Data/CorpusNGT/gloss/"
glossvideopath = viddir_name + "/"
ext = 'mpg'

gloss_counter = dict([(i, 0) for i in c_unique_glosses]) # start counting the glosses so that we can give each split video a number so they don't overwrite eachother
error_count = 0

for filename in tqdm(os.listdir(datapath)):
    file, extension = filename.split('.')
    if extension == ext:
        gloss = pd.read_csv((datapath + file + ".csv")) #, header=None, names = ['signer', 'start seconds', 'start time', 'stop seconds', 'stop time', 'duration', 'value', 'gloss'])
        gloss['gloss'].replace('', np.nan, inplace=True)
        gloss.dropna(subset = ['gloss'], inplace = True)
        gloss = gloss.dropna(axis=0)
        gloss = gloss.iloc[1:]
        gloss = gloss.reset_index(drop=True)
        try:
            split_cut(datapath, glossvideopath, file, ext, gloss['start seconds'], gloss['stop seconds'], gloss['gloss'], gloss_counter)
        except Exception as err:
            error_count += 1
            print(Exception, err, error_count)

## We've now done it only for the videos with just glosses, we do the same for the videos with glosses and translations

#datapath = "drive/My Drive/Thesis/Data/ECHO/gloss_trans/"

## We keep counting where we've been counting from so no need to set gloss counter to 0

#for filename in os.listdir(datapath):
#    file, extension = filename.split('.')
#    if extension == ext:
#        print(file)
#        gloss = pd.read_csv((datapath + file + ".csv"), header=None, names = ['start seconds', 'start time', 'stop seconds', 'stop time', 'duration', 'value', 'gloss'])
#        gloss = gloss.dropna(axis=0)
#        gloss = gloss.loc[["Gloss RH", "Gloss LH"]]
#        gloss = gloss[~gloss.gloss.str.contains("(p-)")] ## removes unfinished annotations
#        gloss = gloss[~gloss.gloss.str.contains("(g-)")] ## removes unfinished annotations
#        gloss['gloss'] = gloss['gloss'].apply(lambda x: cleanItem(x)) ## cleans up the glosses like we did with the statisitcs

#        split_cut(datapath, glossvideopath, file, ext, gloss['start time'], gloss['stop time'], gloss['gloss'], gloss_counter)

  0%|          | 0/1208 [00:00<?, ?it/s]

Input #0, mpeg, from 'Data/CorpusNGT/gloss/CNGT0004_S003.mpg':
  Duration: 00:05:41.28, start: 0.187533, bitrate: 1730 kb/s
    Stream #0:0[0x1e0]: Video: mpeg1video, yuv420p(tv), 352x288 [SAR 178:163 DAR 1958:1467], 1500 kb/s, 25 fps, 25 tbr, 90k tbn, 25 tbc
    Stream #0:1[0x1c0]: Audio: mp2, 48000 Hz, mono, s16p, 192 kb/s
At least one output file must be specified



  0%|          | 2/1208 [00:43<7:15:11, 21.65s/it]

Input #0, mpeg, from 'Data/CorpusNGT/gloss/CNGT0004_S004.mpg':
  Duration: 00:05:41.28, start: 0.187533, bitrate: 1730 kb/s
    Stream #0:0[0x1e0]: Video: mpeg1video, yuv420p(tv), 352x288 [SAR 178:163 DAR 1958:1467], 1500 kb/s, 25 fps, 25 tbr, 90k tbn, 25 tbc
    Stream #0:1[0x1c0]: Audio: mp2, 48000 Hz, mono, s16p, 192 kb/s
At least one output file must be specified



  0%|          | 4/1208 [01:08<5:28:08, 16.35s/it]

...

100%|█████████▉| 1204/1208 [2:29:00<00:12,  3.05s/it]

Input #0, mpeg, from 'Data/CorpusNGT/gloss/CNGT2272_S090.mpg':
  Duration: 00:02:08.00, start: 0.194911, bitrate: 1666 kb/s
    Stream #0:0[0x1e0]: Video: mpeg1video, yuv420p(tv), 352x288 [SAR 178:163 DAR 1958:1467], 1500 kb/s, 25 fps, 25 tbr, 90k tbn, 25 tbc
    Stream #0:1[0x1c0]: Audio: mp2, 48000 Hz, stereo, s16p, 128 kb/s
At least one output file must be specified



100%|█████████▉| 1206/1208 [2:29:02<00:04,  2.44s/it]

Input #0, mpeg, from 'Data/CorpusNGT/gloss/CNGT2276_S090.mpg':
  Duration: 00:01:05.00, start: 0.194911, bitrate: 1667 kb/s
    Stream #0:0[0x1e0]: Video: mpeg1video, yuv420p(tv), 352x288 [SAR 178:163 DAR 1958:1467], 1500 kb/s, 25 fps, 25 tbr, 90k tbn, 25 tbc
    Stream #0:1[0x1c0]: Audio: mp2, 48000 Hz, stereo, s16p, 128 kb/s
At least one output file must be specified



100%|██████████| 1208/1208 [2:29:03<00:00,  7.40s/it]


In [45]:
np.save('Data/CorpusNGT/gloss_counts.npy', gloss_counter)

In [46]:
## Here I connect every translation to every gloss for the translated videos and save that as the new csv data file

datapath = "Data/ECHO/gloss_trans/"
outputpath = "Data/ECHO/"
ext = 'csv'

def check_interval(startgloss, stopgloss, startsent, stopsent):
    if startgloss >= startsent and stopgloss <=stopsent:
        return True
    else:
        return False

total_data_df = pd.DataFrame()

for filename in os.listdir(datapath):
    file, extension = filename.split('.')
    if extension == ext:
        print(file)
        gloss_location = os.path.join(datapath, filename)
        gloss = pd.read_csv(gloss_location, header=None, names = ['start seconds', 'start time', 'stop seconds', 'stop time', 'duration', 'value', 'item'])
        gloss = gloss.dropna(axis=0)
        gloss = gloss[~gloss.item.str.contains("(p-)")] ## removes unfinished annotations
        gloss = gloss[~gloss.item.str.contains("(g-)")] ## removes unfinished annotations
        gloss.rename(index={"Translation Dutch" : "Translation"},inplace=True)
        gloss.rename(index={"Translation Dutch " : "Translation"},inplace=True)
        gloss.rename(index={"Dutch Translation" : "Translation"},inplace=True)
        gloss.rename(index={"Dutch Translation " : "Translation"},inplace=True)
        sentences = gloss.loc[["Translation"]]                                  # copy dataframe with just translations to seperate df
        sentences['gloss'] = sentences.apply(lambda x: list(), axis=1)          # create another column with empty lists, we will add the glosses to this

        gloss = gloss.loc[["Gloss RH Dutch", "Gloss LH Dutch"]]
        gloss['item'] = cleanItem(gloss['item'])           # cleans up the glosses like we did with the statisitcs

        for i, sentence in sentences.iterrows():
            for j, glossrow in gloss.iterrows():
                if check_interval(glossrow['start time'], glossrow['stop time'], sentence['start time'], sentence['stop time']):
                    item = glossrow['item']
                    sentence['gloss'].append(item)


        total_data_df = pd.concat([total_data_df, sentences], ignore_index=False, sort=False)


NGT_AH_fab1
NGT_AH_fab2
NGT_AH_fab3
NGT_AH_fab4
NGT_AH_fab5
NGT_AW_fab1
NGT_AW_fab2
NGT_AW_fab3
NGT_AW_fab4
NGT_AW_fab5
NGT_JR_fab1
NGT_JR_fab2
NGT_JR_fab3
NGT_JR_fab4
NGT_JR_fab5
NGT_WE_poems


In [47]:
import string

total = total_data_df[total_data_df['gloss'].map(len) > 0]
total = total.rename(columns={"item": "sentence"})
total = total.reset_index(drop=True)

total['sentence'] = total['sentence'].apply(lambda x: (x.lower().translate(str.maketrans('', '', string.punctuation)) ))

total = total.drop(['start seconds',	'start time',	'stop seconds',	'stop time',	'duration',	'value'], axis=1)

In [48]:
total.tail(10)

## After removing the entries with no glosses, we are left with just 483 sentences

,sentence,gloss
474,de kwaliteit bewijst het,"[BEWIJS, BEWIJS]"
475,dus laten we altijd zo verder gaan,"[LEKKER, RICHTING-OPGAAN]"
476,de aarde draait,[AARDE-DRAAIEN]
477,donkerte,"[VALLEN-VIA-HART, VALLEN-VIA-HART]"
478,innerlijke strijd,"[INNERLIJKE-STRIJD, INNERLIJKE-STRIJD]"
479,chaos,"[INNERLIJKE-CHAOS, INNERLIJKE-CHAOS]"
480,donkerte,"[VALLEN-VIA-HART, VALLEN-VIA-HART]"
481,yinyang,"[YIN-YANG, YIN-YANG]"
482,een bonzend hart,"[YIN-YANG-IN-HART-KLOPPEN, YIN-YANG-IN-HART-KL..."
483,de zon gaat langzaam onder,"[ZON-ONDERGAAN, AARDE]"


In [49]:
# Save the dataframe to csv
total.to_csv('Data/ECHO/sentences_with_gloss.csv')



---



Turn each video into frames


In [50]:
def get_frames(filename, n_frames= 1):
    frames = []
    v_cap = cv.VideoCapture(filename)
    v_len = int(v_cap.get(cv.CAP_PROP_FRAME_COUNT))
    frame_list= np.linspace(0, v_len-1, n_frames+1, dtype=np.int16)

    for fn in range(v_len):
        success, frame = v_cap.read()
        if success is False:
            continue
        if (fn in frame_list):
            frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            frames.append(frame)
    v_cap.release()
    return frames, v_len

def store_frames(frames, path2store):
    for ii, frame in enumerate(frames):
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
        path2img = os.path.join(path2store, "frame"+str(ii)+".jpg")
        cv.imwrite(path2img, frame)

In [51]:
from tqdm import tqdm

path2data = viddir_name

c_unique_glosses = np.load('Data/CorpusNGT/gloss_labels.npy',allow_pickle='TRUE').item()
#gloss_counter = np.load('Data/CorpusNGT/gloss_counts.npy',allow_pickle='TRUE').item()

for item in tqdm(c_unique_glosses):
    try:
        sub_folder = item
        path2gloss = os.path.join(path2data, sub_folder)
    
        listOfCategories = os.listdir(path2gloss)
        listOfCategories, len(listOfCategories)
    
        extension = ".mpg"
        n_frames = 8
        for root, dirs, files in os.walk(path2gloss, topdown=False):
            try:
                for name in files:
                    try:
                        if extension not in name:
                            continue
                        path2vid = os.path.join(root, name)
                        frames, vlen = get_frames(path2vid, n_frames= n_frames)
                        path2store = path2vid.replace(extension, "")
                        #print(path2store)
                        os.makedirs(path2store, exist_ok= True)
                        store_frames(frames, path2store)
                    except:
                        print(extension + " not in name")
            except:
                print(name + " not in files")
    except:
        print(path2gloss + " not found in directory")

 54%|█████▎    | 2024/3781 [1:01:52<08:08,  3.60it/s] 

Data/CorpusNGT/gloss_split_8frames\DRAW\R-4 not found in directory


100%|██████████| 3781/3781 [1:05:50<00:00,  1.04s/it]


In [4]:
from tqdm import tqdm
import pandas as pd
import os

totalcount = 0
removecount = 0
removelist = []
for catg in tqdm(os.listdir(viddir_name)):
    f1 = '{}/{}'.format(viddir_name, catg)
    for number in os.listdir(f1):
        f2 = '{}/{}'.format(f1, number)
        remove = False
        if os.path.isdir(f2):
            if len(os.listdir(f2)) < 9: 
                removecount += 1
                totalcount += 1
                removelist.append(f2)
            else:
                totalcount += 1

print(removecount)
print(totalcount)
print(removecount/totalcount)   ## This is the fraction of the data we have to remove because there were not enough frames 
                                ## in the sign to reach the 8 frame minimum

100%|██████████| 3780/3780 [00:19<00:00, 191.13it/s]

15696
116227
0.13504607363177232


In [9]:
## So now we remove these entries that didn't have enough frames, after which the pose estimation should work!

for path in tqdm(removelist):
    if len(os.listdir(path)) != 0:
        for item in os.listdir(path):
            os.remove('{}/{}'.format(path, item))
    os.rmdir(path)

## Stop Here and go run the MediaPipe_Data 

## Fixing the Train/Test/Val Split CorpusNGT

In [73]:
import os
import shutil
import pandas as pd
import numpy as np
from numpy import asarray
import cv2 as cv
import re
import math
from subprocess import check_call, PIPE, Popen
import shlex
import glob
from PIL import Image
import random

import torchvision.transforms as transforms
from PIL import Image
import torch
from torchvision import models
from torch import nn
from torch.utils.data import Dataset, DataLoader, Subset

import torchvision.transforms as transforms
from torchvision.transforms.functional import to_pil_image
import matplotlib.pylab as plt

np.random.seed(2024)
random.seed(2024)
torch.manual_seed(2024)

In [74]:
path2data = "Data/CorpusNGT/gloss_split_8frames/"

listOfCategories = os.listdir(path2data)
listOfCategories, len(listOfCategories)

unique_glosses = np.load('Data/CorpusNGT/gloss_labels.npy',allow_pickle='TRUE').item()
gloss_counter = np.load('Data/CorpusNGT/gloss_counts.npy',allow_pickle='TRUE').item()
print(len(unique_glosses))

3781


In [118]:
cutoff = 50  ## You can change the cutoff and upper limit values to see how that affects the results.
upperlimit = 2000  ## Run the code from here and change these values to save the different cutoffs

def get_vids(path2data):
    listOfCats = os.listdir(path2data)
    ids = []
    labels = []
    usableListOfCats = []
    for catg in listOfCats:
        path2catg = os.path.join(path2data, catg)
        counter = 0
        nr = len(os.listdir(path2catg))
        if nr >= cutoff and counter <=upperlimit:  # We can only use glosses which have more than 4 occurences, meaning more than 2*2 files in dir, since video and frames are in there.
            counter += 1
            listOfSubCats = os.listdir(path2catg)
            path2subCats = [os.path.join(path2catg, los) for los in listOfSubCats if os.path.isdir(os.path.join(path2catg, los))]
            path2subCats = [path for path in path2subCats if len(os.listdir(path)) > 8]  
            #9 because there are 8 frames and one keypoints object, and we want to make sure there is a keypoints object
            if len(path2subCats) >= cutoff: ## check that there's still at least 10 paths after removing outliers, enough for a test, training, val split that goes 1:9:1
                ids.extend(path2subCats)
                labels.extend([catg]*len(path2subCats))
                usableListOfCats.append(catg)
    return ids, labels, usableListOfCats

all_vids, all_labels, catgs = get_vids(path2data)
len(all_vids), len(all_labels), len(catgs)

(74840, 74840, 344)

In [119]:
labels_dict = {}
ind = 0
for uc in catgs:
    labels_dict[uc] = ind
    ind+=1

num_classes = len(catgs)
unique_ids = [id_ for id_, label in zip(all_vids,all_labels) if labels_dict[label]<num_classes]
unique_labels = [label for id_, label in zip(all_vids,all_labels) if labels_dict[label]<num_classes]
len(unique_ids), len(unique_labels)

(74840, 74840)

In [120]:
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [121]:
df = pd.DataFrame(unique_ids, unique_labels)
df = df.reset_index()
df = df.rename(columns={"index": "id", 0: "path"})

In [122]:
train, validate, test = train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=0)

In [123]:
train.size / validate.size

8.0

In [124]:
train.head()

,id,path
6663,CI-A,Data/CorpusNGT/gloss_split_8frames/CI-A\41
38415,PO,Data/CorpusNGT/gloss_split_8frames/PO\2284
17294,HANDEN-WRIJVEN-B,Data/CorpusNGT/gloss_split_8frames/HANDEN-WRIJ...
20835,JA-A,Data/CorpusNGT/gloss_split_8frames/JA-A\974
55764,PT,Data/CorpusNGT/gloss_split_8frames/PT\3410


In [125]:
print(train.size+validate.size+test.size)

149680


In [126]:
train.to_pickle("Data/CorpusNGT/train_cutoff{}_upperlimit{}.pkl".format(cutoff, upperlimit))
test.to_pickle("Data/CorpusNGT/test_cutoff{}_upperlimit{}.pkl".format(cutoff, upperlimit))
validate.to_pickle("Data/CorpusNGT/validate_cutoff{}_upperlimit{}.pkl".format(cutoff, upperlimit))

You're done! Go train the model now

----------------------------

In [3]:
### Extra code for the SURF server
import os
import glob

for p in glob.glob('Data/CorpusNGT/gloss_split_8frames/**/*.MPG', recursive=True):
    if os.path.isfile(p):
        os.remove(p)

In [2]:
### Extra code to zip the data and upload it to SURF server
import shutil
shutil.make_archive("CSL-Daily", 'zip', "Data/CSL-Daily")

'C:\\Users\\novan\\Desktop\\Thesis\\CSL-Daily.zip'

In [3]:
filename = 'gloss_split_8frames_zipped2.zip'
zipdir = 'gloss_split_8frames'
shutil.unpack_archive(filename, zipdir, "zip")  

KeyboardInterrupt: 